In [118]:
import nibabel as nib
import numpy as np
import pandas as pd

class Atlas:
    def __init__(self, filepath, csv_key_path=None):
        """Init function for Atlas class.
        Args:
            filepath (str): Path to the .nii.gz file of the atlas.
            csv_key_path (str): Path to the .csv file containing the key for the atlas. 
            If not provided, the default is to look for a .csv file with the same name as the .nii.gz file in the parent directory of the .nii.gz file."""
        
        self.filepath = filepath
        if csv_key_path:
            self.csv_key_path = csv_key_path
        else:
            self.csv_key_path = str(filepath.split(".")[0] + ".csv")
        self.key = pd.read_csv(self.csv_key_path)
        self.atlas = nib.load(self.filepath)
    
    def name_at_index(self, index=[48, 94, 35]):
        """Returns the name of the region at the given index.
        Args:
            index (list): List of three integers representing the x, y, z coordinates of the index (in voxel space).
        Returns:
            str: Name of the region at the given index."""
        
        value_at_index = self.atlas.get_fdata()[tuple(index)]
        matched_row = self.key[self.key['value'] == value_at_index]
        if len(matched_row) == 1:
            return matched_row['name'].iloc[0]
        elif len(matched_row) > 1:
            return matched_row['name'].tolist()
        else:
            return "No matching region found"

class NiftiLabeler:
    def __init__(self, atlas, nifti, min_threshold=1, max_threshold=1):
        """Init function for NiftiLabeler class.
        Args:
            atlas (Atlas): An Atlas object, or the path to the .nii.gz file of the atlas.
            nifti (str): Nifti obj, or path to the .nii.gz file of the nifti volume to be labeled.
            min_threshold (int): Minimum value to be considered as a label in the nifti volume.
            max_threshold (int): Maximum value to be considered as a label in the nifti volume.
        """

        if type(atlas) != Atlas:
            print(f"Looking for atlas {atlas}...")
            try:
                atlas = Atlas(atlas)
            except:
                print(f"""Could not find atlas {atlas}. Make sure the path is correct and try again. 
                      \n Example: atlases/HarvardOxford-cort-maxprob-thr0-2mm.nii.gz""")
                return
        self.atlas = atlas

        if type(nifti) != nib.nifti1.Nifti1Image:
            print(f"Looking for nifti {nifti}...")
            try:
                nifti = nib.load(nifti)
            except:
                print(f"""Could not find nifti {nifti}. Make sure the path is correct and try again. 
                      \n Example: volumes/30lechowiczglogowska2019.nii.gz""")
                return
        self.nifti = nifti
        self.volume_data = nifti.get_fdata()
        self.labeled_data = None
        self.unique_labels = None
        self.min_threshold = min_threshold
        self.max_threshold = max_threshold
    
    def label_volume(self):
        """Labels the nifti volume with the names of the regions from the atlas."""
        
        if self.atlas.atlas.shape != self.nifti.shape:
            print(f"The shape of the atlas ({self.atlas.atlas.shape}) and the nifti volume ({({self.nifti.shape})}) do not match. Please provide a nifti volume with the same shape as the atlas.")
            return

        # Find indices where the volume data is within the specified threshold range
        masked_indices = np.where(np.logical_and(self.volume_data >= self.min_threshold, 
                                         self.volume_data <= self.max_threshold))
        # Prepare a dictionary to store the results
        results = {'index': [], 'atlas_label': []}

        # Iterate over the masked indices
        for i, j, k in zip(*masked_indices):
            label = self.atlas.name_at_index([i, j, k])

            # Store the results
            results['index'].append((i, j, k))
            results['atlas_label'].append(label)

        # Convert the results to a DataFrame
        results_df = pd.DataFrame(results)
        self.labeled_data = results_df
        self.unique_labels = results_df['atlas_label'].unique()
        return self

volume_to_label = r'volumes\30lechowiczglogowska2019.nii.gz'
atlas_cort = r"atlases\HarvardOxford-cort-maxprob-thr0-2mm.nii.gz"
atlas_sub = r"atlases\HarvardOxford-sub-maxprob-thr0-2mm.nii.gz"

# Assuming unique_labels from both label_volume() calls are NumPy arrays
unique_labels_cort = NiftiLabeler(atlas_cort, volume_to_label).label_volume().unique_labels
unique_labels_sub = NiftiLabeler(atlas_sub, volume_to_label).label_volume().unique_labels

# Concatenate arrays and then find unique elements
unique_labels = np.unique(np.concatenate((unique_labels_cort, unique_labels_sub)))

unique_labels


Looking for atlas atlases\HarvardOxford-cort-maxprob-thr0-2mm.nii.gz...
Looking for nifti volumes\30lechowiczglogowska2019.nii.gz...
Looking for atlas atlases\HarvardOxford-sub-maxprob-thr0-2mm.nii.gz...
Looking for nifti volumes\30lechowiczglogowska2019.nii.gz...


array(['Cingulate Gyrus, anterior division',
       'Juxtapositional Lobule Cortex (formerly Supplementary Motor Cortex)',
       'Left Cerebral Cortex', 'Left Cerebral White Matter',
       'Middle Frontal Gyrus', 'No matching region found',
       'Superior Frontal Gyrus'], dtype=object)